# My Dinning Spending Habits
The purpose of this notebook is to examine my dinning spending habits during my years in college. I will be using data I receieving from Rit Dining from 08/2021 to 1/2022.

## Imports

In [539]:
import pandas as pd
import plotly.express as px

## Step One: Gathering the Data

In [540]:
august_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-08-01_to_2021-08-31.csv')
september_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-09-01_to_2021-09-30.csv')
october_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-10-01_to_2021-10-31.csv')
november_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-11-01_to_2021-11-30.csv')
december_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-12-01_to_2021-12-14.csv')
january_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-12-16_to_2022-01-31.csv')


In [541]:
august_df.head(3)

,Date,Description,Amount,Balance
0,8/31/2021 06:20PM,Beanz 1 [1505],-5.29,2624.30
1,8/31/2021 05:52PM,Global Village Cantina & Grille 2 [1591],-9.99,2629.59
2,8/31/2021 12:54PM,GOL_1940_69_BEVERAGE,-2.25,2639.58


### Storing the DataFrame into lists catagorized by their semester

In [542]:
fall_semester = [august_df, september_df, october_df, november_df, december_df]
spring_semester = [january_df]
all_semesters = [fall_semester, spring_semester]

## Step Two: Cleaning the Data

### Helper Functions to Clean Up Unneeded Info in the Description Column

In [543]:
def remove_brackets(row):
    return row.split(' [')[0]

def remove_num_at_end(row):
    if row[len(row)-1].isnumeric(): 
        return row[0:len(row)-2]
    else:
        return row

def remove_bad_suffixes(row):
    row = row.removesuffix(' OnDemand')
    row = row.removesuffix(' - Tablet')
    return row

def clean_up_description(row):
    row = remove_brackets(row)
    row = remove_num_at_end(row)
    row = remove_bad_suffixes(row)
    return row

### Helper Function for Creating New Time Columns

In [544]:
day_of_week_map = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

def create_time_columns(t):
    return pd.Series({'Year': t.year, 'Month': t.month, 'Day Of Week': t.day_of_week, 'Hour': t.hour})

### Cleaning Up DataFrames and Combining Them Based on Semester

In [545]:
def clean_semester_data(semester):
    for i in range(0, len(semester)):
        semester[i]['Description'] = semester[i]['Description'].apply(clean_up_description)
        semester[i]['Date'] = pd.to_datetime(semester[i]['Date'])
        semester[i].sort_values(by='Date', inplace=True)
        semester[i] = pd.concat([semester[i], semester[i]['Date'].apply(create_time_columns)], axis=1)
        semester[i]['Day Of Week'] = semester[i]['Day Of Week'].map(day_of_week_map)
        semester[i] = semester[i].reset_index()
        semester[i].drop('index', axis=1, inplace=True)
        semester[i] = semester[i][['Description', 'Amount', 'Balance', 'Date', 'Year', 'Month', 'Day Of Week', 'Hour']]
    return semester

def aggregate_semester_data(semester):
    semester = clean_semester_data(semester)

    semester_df = semester[0]
    for index in range(1, len(semester)):
        semester_df = pd.concat([semester_df, semester[index]])

    semester_df = semester_df.reset_index()
    semester_df.drop('index', axis=1, inplace=True)
    semester_df['Date'] = semester_df['Date'].apply(lambda date: date.date())

    return semester_df

In [546]:
fall_semester_df = aggregate_semester_data(fall_semester)
spring_semester_df = aggregate_semester_data(spring_semester)

## Step Three: Modeling and Analysing the Data

### The Fall Semester

#### Previewing the Fall Semester Data Frame

In [547]:
fall_semester_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Description  300 non-null    object 
 1   Amount       300 non-null    float64
 2   Balance      300 non-null    float64
 3   Date         300 non-null    object 
 4   Year         300 non-null    int64  
 5   Month        300 non-null    int64  
 6   Day Of Week  300 non-null    object 
 7   Hour         300 non-null    int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 18.9+ KB


In [548]:
fall_semester_df

,Description,Amount,Balance,Date,Year,Month,Day Of Week,Hour
0,Deposit,3020.00,3020.00,2021-08-04,2021,8,Wed,9
1,The Commons,-11.91,3008.09,2021-08-22,2021,8,Sun,15
2,Campus Concessions,-13.00,2995.09,2021-08-22,2021,8,Sun,19
3,Campus Concessions,-13.00,2982.09,2021-08-22,2021,8,Sun,20
4,Brick City Cafe,-19.58,2962.51,2021-08-23,2021,8,Mon,12
...,...,...,...,...,...,...,...,...
295,GOL_2942_67_SNACK,-2.35,22.19,2021-12-13,2021,12,Mon,15
296,GOL_2942_67_SNACK,-1.60,20.59,2021-12-13,2021,12,Mon,15
297,Cafe & Market at Crossroads,-8.29,12.30,2021-12-13,2021,12,Mon,19
298,Ctrl Alt DELi,-6.89,5.41,2021-12-14,2021,12,Tue,11


In [549]:
px.line(fall_semester_df, x='Date', y='Balance', title='Balance Over Semester')

#### How Much Did I Spend This Semester

In [550]:
answer = fall_semester_df['Balance'].iloc[0] - fall_semester_df['Balance'].iloc[-1]
print('This semester I spent ${0:.2f} in total'.format(answer))

This semester I spent $3020.00 in total


In [551]:
augTotal = abs(
    fall_semester_df[fall_semester_df['Month'] == 8]['Amount'].iloc[1:].sum())
septTotal = abs(
    fall_semester_df[fall_semester_df['Month'] == 9]['Amount'].sum())
octTotal = abs(
    fall_semester_df[fall_semester_df['Month'] == 10]['Amount'].sum())
novTotal = abs(
    fall_semester_df[fall_semester_df['Month'] == 11]['Amount'].sum())
decTotal = abs(
    fall_semester_df[fall_semester_df['Month'] == 12]['Amount'].sum())

In [552]:
monthSpendingsDf = pd.DataFrame({'Month': ['August', 'September', 'October', 'November', 'December'], 'Total Spent': [
                                augTotal, septTotal, octTotal, novTotal, decTotal]})

fig = px.bar(monthSpendingsDf, x='Month', y='Total Spent',
       title='Money Spent Each Month')
fig.show()

print("""In august I spent ${0:.2f},
In september I spent ${1:.2f},
In october I spent ${2:.2f},
In november I spent ${3:.2f},
In december I spent ${4:.2f}.""".format(augTotal, septTotal, octTotal, novTotal, decTotal))


In august I spent $395.70,
In september I spent $905.08,
In october I spent $706.88,
In november I spent $675.27,
In december I spent $337.07.


In [553]:
import math

monthsDict = {
    8: {'name': 'August', 'num': 8}, 
    9: {'name': 'September', 'num': 9}, 
    10: {'name': 'October', 'num': 10}, 
    11: {'name': 'November', 'num': 11}, 
    12: {'name': 'December', 'num': 12}
    }


def getDataFrameSubset(month_dict):
    monthDf = fall_semester_df[fall_semester_df['Month']
                               == month_dict['num']].iloc[0:]
    if month_dict['num'] == 8:
        monthDf = fall_semester_df[fall_semester_df['Month']
                                == month_dict['num']].iloc[1:]
    return monthDf


def groupByDate(monthDf):
    monthDf['Amount'] = monthDf['Amount'].apply(lambda money: abs(money))
    monthSpendingsPerDay = monthDf.groupby(by=['Date'])['Amount'].sum()
    return monthSpendingsPerDay


def createGraph(monthSpendingsPerDay, month_dict):
    fig = px.bar(monthSpendingsPerDay, y='Amount',
                 title='Money Spent Per Day in {0}'.format(month_dict['name']))
    fig.show()


def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier


def getMeanSpending(month_num):
    monthDf = getDataFrameSubset(monthsDict[month_num])
    monthSpendingsPerDay = groupByDate(monthDf)
    return round_up(monthSpendingsPerDay.mean(), 2)


def spendingInMonthPerDay(month_dict):
    monthDf = getDataFrameSubset(month_dict)
                                
    monthSpendingsPerDay = groupByDate(monthDf)

    createGraph(monthSpendingsPerDay, month_dict)

    print('On average in {0} I spent ${1:.2f} per day over {2} days'.format(
        month_dict['name'], monthSpendingsPerDay.mean(), monthSpendingsPerDay.count()))

In [554]:
spendingInMonthPerDay(monthsDict[8])

On average in August I spent $39.57 per day over 10 days


In [555]:
spendingInMonthPerDay(monthsDict[9])

On average in September I spent $31.21 per day over 29 days


In [556]:
spendingInMonthPerDay(monthsDict[10])

On average in October I spent $26.18 per day over 27 days


In [557]:
spendingInMonthPerDay(monthsDict[11])

On average in November I spent $28.14 per day over 24 days


In [558]:
spendingInMonthPerDay(monthsDict[11])

On average in November I spent $28.14 per day over 24 days


In [559]:
avgSpendingEachMonth = []

for month in monthsDict:
    avgSpendingEachMonth.append(getMeanSpending(month))

avgSpendingEachMonth

monthSpendingsDf['Average Spent Per Day'] = avgSpendingEachMonth
monthSpendingsDf

,Month,Total Spent,Average Spent Per Day
0,August,395.70,39.58
1,September,905.08,31.21
2,October,706.88,26.19
3,November,675.27,28.14
4,December,337.07,24.08


In [560]:
fig = px.bar(monthSpendingsDf, x='Month', y='Average Spent Per Day',
             title='Average Money Spent Per Day Each Month')
fig.show()


In [561]:
def figures_to_html(figs, filename="dashboard.html"):
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")
fig1 = px.bar(monthSpendingsDf, x='Month', y='Average Spent Per Day',
             title='Average Money Spent Per Day Each Month')
fig2 = px.bar(monthSpendingsDf, x='Month', y='Average Spent Per Day',
             title='Average Money Spent Per Day Each Month')
figures_to_html([fig1, fig2])

##### Conclusion

After looking at the top level of my spending habits I realized that I spend the most in the second month

### The Spring Semester

In [562]:
spring_semester_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Description  24 non-null     object 
 1   Amount       24 non-null     float64
 2   Balance      24 non-null     float64
 3   Date         24 non-null     object 
 4   Year         24 non-null     int64  
 5   Month        24 non-null     int64  
 6   Day Of Week  24 non-null     object 
 7   Hour         24 non-null     int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 1.6+ KB


In [563]:
spring_semester_df.head()

,Description,Amount,Balance,Date,Year,Month,Day Of Week,Hour
0,Deposit,3020.00,3020.00,2021-12-16,2021,12,Thu,10
1,The Commons,-10.18,3009.82,2022-01-09,2022,1,Sun,12
2,Brick City Cafe,-9.67,3000.15,2022-01-10,2022,1,Mon,7
3,Brick City Cafe,-3.60,2996.55,2022-01-10,2022,1,Mon,7
4,RITz Sports Zone,-9.99,2986.56,2022-01-11,2022,1,Tue,16
